In [45]:
## ライブラリ
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import requests
import jpy_datareader.data as web

In [58]:
## 自前関数
'''
東京オープデータAPIデータ取得用関数
url : https://spec.api.metro.tokyo.lg.jp/spec/search
'''
def apiJsonData(url,limit):
    # クエリパラメーター
    params = {
        'limit': limit
        }
    # POSTデータ（空のJSONオブジェクト）
    data = {}
    # ヘッダー
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
    }
    # POSTリクエストを送信してレスポンスを取得
    return requests.post(url, params=params, json=data, headers=headers)

'''
全件取得
'''
def apiJsonDataAll(url):
    # クエリパラメーター
    params = {
        'limit': 1
        }
    # POSTデータ（空のJSONオブジェクト）
    data = {}
    # ヘッダー
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
    }
    # POSTリクエストを送信してレスポンスを取得
    response = requests.post(url, params=params, json=data, headers=headers)
    if response.status_code == 200:
        json_data = response.json()
        total = json_data["total"]
        limit = json_data["limit"]
        if (total > limit) or (total < 500):
            time.sleep(1)
            response = apiJsonData(url,total)
    return response

'''
東京オープデータJSONからデータフレームに変換する関数
'''
def jsonToDataFrame(response):
    json_data = response.json()
    df = pd.DataFrame(json_data['hits'])
    return df



In [48]:
url = 'https://service.api.metro.tokyo.lg.jp/api/t132098d0000000103-80b9034ec2419cfeec601232cb3ff280-0/json'
response = apiJsonData(url,1)
# レスポンスのステータスコードをチェック
if response.status_code == 200:
	# JSONデータを取得
	json_data = response.json()
	total = json_data["total"]
	limit = json_data["limit"]
	if (total > limit) or (total < 500):
		#アクセス負担の軽減策
		time.sleep(1)
		response = apiJsonData(url,total)
		json_data = response.json()
	print(f'OK({response})')
	print(f'データ数:{json_data["limit"]}/{json_data["total"]}')
else:
	print('取得失敗')

OK(<Response [200]>)
データ数:279/279


In [47]:
json_data

{'total': 279,
 'subtotal': 279,
 'limit': 279,
 'offset': None,
 'metadata': {'apiId': 't132098d0000000103-80b9034ec2419cfeec601232cb3ff280-0',
  'title': 'AED設置箇所一覧 AED設置箇所一覧',
  'datasetId': 't132098d0000000103',
  'datasetTitle': 'AED設置箇所一覧',
  'datasetDesc': '【町田市】町田市が管理する市内公共施設のAED設置箇所一覧です。本データは、内閣官房情報通信技術(IT)総合戦略室が公開している標準データセットの様式をもとに作成しています。\n提供する情報は2019年10月1日時点のものです。\n本データは、地図作成上の誤差を含んでいます。また、データ上の表示と現況とは異なる場合があります。\n',
  'dataTitle': 'AED設置箇所一覧',
  'dataDesc': '',
  'sheetname': '',
  'version': '1.0.1',
  'created': '2019/12/26 00:00:00',
  'updated': '2021/12/07 00:00:00'},
 'hits': [{'row': 1,
   '都道府県コード又は市区町村コード': 132098,
   'NO': 100001,
   '都道府県名': '東京都',
   '市区町村名': '町田市',
   '名称': '町田市庁舎',
   '名称_カナ': 'マチダシチョウシャ',
   '住所': '東京都町田市森野2-2-22',
   '方書': '',
   '緯度': 35.54663719,
   '経度': 139.4386429,
   '設置位置': '1F南入口付近',
   '電話番号': '042-722-3111',
   '内線番号': '',
   '法人番号': 6000020132098,
   '団体名': '町田市',
   '利用可能曜日': '月火水木金',
   '開始時間': '08:30',
   '終了時間': '17:00',
   

In [49]:
df = pd.DataFrame(json_data['hits'])
df.columns

Index(['row', '都道府県コード又は市区町村コード', 'NO', '都道府県名', '市区町村名', '名称', '名称_カナ', '住所',
       '方書', '緯度', '経度', '設置位置', '電話番号', '内線番号', '法人番号', '団体名', '利用可能曜日',
       '開始時間', '終了時間', '利用可能日時特記事項', '小児対応設備の有無', 'URL', '備考'],
      dtype='object')

In [50]:
df[['都道府県名','市区町村名','名称','団体名','緯度','経度','小児対応設備の有無']]

,都道府県名,市区町村名,名称,団体名,緯度,経度,小児対応設備の有無
0,東京都,町田市,町田市庁舎,町田市,35.546637,139.438643,有
1,東京都,町田市,町田市庁舎,町田市,35.546637,139.438643,有
2,東京都,町田市,町田市庁舎,町田市,35.546637,139.438643,有
3,東京都,町田市,町田市庁舎,町田市,35.546637,139.438643,有
4,東京都,町田市,町田市庁舎,町田市,35.546637,139.438643,有
...,...,...,...,...,...,...,...
274,東京都,町田市,市立室内プール(健康増進温浴施設),町田市,35.586280,139.413823,有
275,東京都,町田市,鶴間公園,町田市,35.510058,139.468465,有
276,東京都,町田市,町田薬師池公園四季彩の杜西園,町田市,35.574804,139.446521,有
277,東京都,町田市,小山子どもクラブ,町田市,35.595127,139.371799,有


In [51]:
df2 = df[['都道府県名','市区町村名','名称','団体名','緯度','経度','小児対応設備の有無']]
# 重複する行を削除して一意の行のみを残す
df2.drop_duplicates()

,都道府県名,市区町村名,名称,団体名,緯度,経度,小児対応設備の有無
0,東京都,町田市,町田市庁舎,町田市,35.546637,139.438643,有
5,東京都,町田市,町田市民フォーラム,町田市,35.541772,139.449944,有
6,東京都,町田市,成瀬コミュニティセンター,町田市,35.543620,139.472881,有
7,東京都,町田市,木曽森野コミュニティセンター,町田市,35.555100,139.428546,有
8,東京都,町田市,つくし野コミュニティセンター,町田市,35.527098,139.484676,有
...,...,...,...,...,...,...,...
274,東京都,町田市,市立室内プール(健康増進温浴施設),町田市,35.586280,139.413823,有
275,東京都,町田市,鶴間公園,町田市,35.510058,139.468465,有
276,東京都,町田市,町田薬師池公園四季彩の杜西園,町田市,35.574804,139.446521,有
277,東京都,町田市,小山子どもクラブ,町田市,35.595127,139.371799,有


In [52]:
df2.to_csv('../data/matida.csv',index=True)

In [59]:
url = 'https://service.api.metro.tokyo.lg.jp/api/t131083d0000000027-d26151caf1370dd6c31d8931a2d8d189-0/json'
response = apiJsonDataAll(url)
# レスポンスのステータスコードをチェック
if response.status_code == 200:
	# JSONデータを取得
	json_data = response.json()
	total = json_data["total"]
	limit = json_data["limit"]
	print(f'OK({response})')
	print(f'データ数:{json_data["limit"]}/{json_data["total"]}')
else:
	print('取得失敗')
df = pd.DataFrame(json_data['hits'])
df.columns


OK(<Response [200]>)
データ数:246/246


Index(['row', '都道府県コード又は市区町村コード', 'NO', '都道府県名', '市区町村名', '名称', '名称_カナ', '住所',
       '方書', '緯度', '経度', '設置位置', '電話番号', '内線番号', '法人番号', '団体名', '利用可能曜日',
       '開始時間', '終了時間', '利用可能日時特記事項', '小児対応設備の有無', 'URL', '備考'],
      dtype='object')

In [38]:
df2 = df[['都道府県名','市区町村名','名称','団体名','緯度','経度','小児対応設備の有無']]
# 重複する行を削除して一意の行のみを残す
df2.drop_duplicates()

,都道府県名,市区町村名,名称,団体名,緯度,経度,小児対応設備の有無
0,東京都,江東区,有明西学園,,35.637038,139.784381,
1,東京都,江東区,有明スポーツセンター,,35.633206,139.783810,
2,東京都,江東区,有明小学校,,35.640466,139.795945,
3,東京都,江東区,有明中学校,,35.640432,139.795970,
4,東京都,江東区,扇橋小学校,,35.681289,139.811726,
...,...,...,...,...,...,...,...
241,東京都,江東区,夢の島総合運動場,,35.650166,139.823387,
242,東京都,江東区,夢の島スケートボードパーク,,35.654260,139.822598,
243,東京都,江東区,夢の島競技場,,35.648514,139.823658,
244,東京都,江東区,夢の島区民農園,,35.647474,139.822650,


In [39]:
df2.to_csv('../data/koutouku.csv',index=True)

In [42]:
## 縦結合
df1 = pd.read_csv('../data/koutouku.csv',index_col=0)
df2 = pd.read_csv('../data/matida.csv',index_col=0)
## concatで結合
## ignore_index=True:インデックス付け直し
df3 = pd.concat([df1,df2], ignore_index=True)
df3

,都道府県名,市区町村名,名称,団体名,緯度,経度,小児対応設備の有無
0,東京都,江東区,有明西学園,NaN,35.637038,139.784381,NaN
1,東京都,江東区,有明スポーツセンター,NaN,35.633206,139.783810,NaN
2,東京都,江東区,有明小学校,NaN,35.640466,139.795945,NaN
3,東京都,江東区,有明中学校,NaN,35.640432,139.795970,NaN
4,東京都,江東区,扇橋小学校,NaN,35.681289,139.811726,NaN
...,...,...,...,...,...,...,...
520,東京都,町田市,市立室内プール(健康増進温浴施設),町田市,35.586280,139.413823,有
521,東京都,町田市,鶴間公園,町田市,35.510058,139.468465,有
522,東京都,町田市,町田薬師池公園四季彩の杜西園,町田市,35.574804,139.446521,有
523,東京都,町田市,小山子どもクラブ,町田市,35.595127,139.371799,有


In [43]:
# 小児対応設備の有無のNaNを無に変更
df4 = df3.fillna({'小児対応設備の有無': '無'})
df4

,都道府県名,市区町村名,名称,団体名,緯度,経度,小児対応設備の有無
0,東京都,江東区,有明西学園,NaN,35.637038,139.784381,無
1,東京都,江東区,有明スポーツセンター,NaN,35.633206,139.783810,無
2,東京都,江東区,有明小学校,NaN,35.640466,139.795945,無
3,東京都,江東区,有明中学校,NaN,35.640432,139.795970,無
4,東京都,江東区,扇橋小学校,NaN,35.681289,139.811726,無
...,...,...,...,...,...,...,...
520,東京都,町田市,市立室内プール(健康増進温浴施設),町田市,35.586280,139.413823,有
521,東京都,町田市,鶴間公園,町田市,35.510058,139.468465,有
522,東京都,町田市,町田薬師池公園四季彩の杜西園,町田市,35.574804,139.446521,有
523,東京都,町田市,小山子どもクラブ,町田市,35.595127,139.371799,有
